In [1]:
import numpy as np
import datetime as dt
import logging
import seaborn as sns
import matplotlib.pyplot as plt
from import_sql import *

## Run Parameters

In [2]:
recreate_base_tables=False
#rolling window
n=6
# params to group by
dims=['ACCOUNT_CLASS',
        'FIRST_PRODUCT_CATEGORY',
        # 'PS_INDICATOR',
        'COHORT_YM',
        'COUNTRY'
]

In [3]:
if recreate_base_tables:
    # create table of fees from MP. SQL from Temma
    create_sf_table(create_kpi_mp_fee_text)
    # create table of KPI base, no summarization
    create_sf_table(create_kpi_base_text)
    # create dimension table for ease of joins
    ## don't need if analyzing in pandas
    create_sf_table(create_kpi_dim_text)

In [4]:
# get the data at a month, account class, product, cohort year, country level
kpi_b = get_sf_data('SELECT * FROM PROD_ANALYTICS.PRODUCT_ANALYTICS.KPI_BASE')
kpi_b['ACTIVITY_DATE'] = pd.to_datetime(kpi_b['ACTIVITY_DATE'])
kpi_b['FIRST_CLOSEDATE'] = pd.to_datetime(kpi_b['FIRST_CLOSEDATE'])

Table Fetched


---
## Account Rollups

In [5]:
# Resample to monthly frequency and mark each user as active
monthly_active = kpi_b.groupby(by=['ACCOUNTID']+dims).resample('MS', on='ACTIVITY_DATE').size().unstack(fill_value=0)
monthly_active = (monthly_active > 0).astype(int)  # Mark the user as active (1) or not (0) for each month

## Retained Accounts

In [6]:
# Calculate rolling 12-month activity window, excluding the current month
## Have to transpose and back to using rolling across colums.
## Max makes it one or zero, instead of summing
monthly_rollup = monthly_active.T.rolling(window=n+1, min_periods=1).max().shift(0).T #do we want the shift here? exclude current month?
# get a yearly rollup for visualizations
yearly_rollup = monthly_rollup.T.resample('YS').max().T

In [7]:
x=yearly_rollup.groupby(by=dims).sum().T
retained=x.melt(ignore_index=False, value_name='RETAINED_ACCOUNTS')
retained['RETAINED_ACCOUNTS']=retained.RETAINED_ACCOUNTS.astype(int)
del x

## Churned Accounts

In [8]:
churned_users=yearly_rollup.apply(
    lambda x: [1 if x[col] == 0 and x.shift(periods=1,axis=0)[col]==1 else 0 for col in yearly_rollup.columns], axis=1)
yearly_churn=pd.DataFrame(churned_users.to_list(), index=yearly_rollup.index, columns=yearly_rollup.columns)

In [9]:
x=yearly_churn.groupby(by=dims).sum().T
churned=x.melt(ignore_index=False, value_name='CHURNED_ACCOUNTS')
del x

## New Accounts

In [10]:
# take monthly active, and make =1 if it's the first occurence of activity
new_users=yearly_rollup.apply(
    lambda x: [1 if x[col]==1 and col==x.idxmax() else 0 for col in yearly_rollup.columns], axis=1)
# make a dataframe with same dimensions as monthly rollup
yearly_new=pd.DataFrame(new_users.to_list(), index=yearly_rollup.index, columns=yearly_rollup.columns)

In [11]:
x=yearly_new.groupby(by=dims).sum().T
new=x.melt(ignore_index=False, value_name='NEW_ACCOUNTS')
del x

In [12]:
# new['NEW_ACCOUNTS_ROLLING']=new.NEW_ACCOUNTS.rolling(window=n, min_periods=1).sum().astype(int)

### Combine Them

In [13]:
qout=retained.merge(
    churned, on=['ACTIVITY_DATE']+dims, how='left'
    ).merge(
        new, on=['ACTIVITY_DATE']+dims, how='left')
# qout.info()

In [14]:
qout['RETAINED ACCOUNTS (ROLLING WINDOW, NO NEW)']=qout.RETAINED_ACCOUNTS - qout.NEW_ACCOUNTS

In [15]:
qout.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4430 entries, 2015-01-01 to 2024-01-01
Data columns (total 8 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   ACCOUNT_CLASS                               4430 non-null   object
 1   FIRST_PRODUCT_CATEGORY                      4430 non-null   object
 2   COHORT_YM                                   4430 non-null   object
 3   COUNTRY                                     4430 non-null   object
 4   RETAINED_ACCOUNTS                           4430 non-null   int64 
 5   CHURNED_ACCOUNTS                            4430 non-null   int64 
 6   NEW_ACCOUNTS                                4430 non-null   int64 
 7   RETAINED ACCOUNTS (ROLLING WINDOW, NO NEW)  4430 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 311.5+ KB


---
## GMV

In [16]:
## Get GMV by index 
gmv_s=kpi_b.groupby(by=['ACCOUNTID']+dims).GMV.sum().fillna(0.00)

### Active

In [17]:
# Monthly rollup will have a '1' for each month that is considered active. This will replace these 1s with the value of the index's GMV
gmv_active_df=yearly_rollup.multiply(gmv_s, axis=0)
# sum across accounts
x=gmv_active_df.groupby(by=dims).sum().T
# melt and calculate retained GMV
retained_gmv=x.melt(ignore_index=False, value_name='RETAINED_GMV')
del x
# retained_gmv.head()

### New

In [18]:
# Monthly rollup will have a '1' for each month that is considered active. This will replace these 1s with the value of the index's GMV
gmv_new_df=yearly_new.multiply(gmv_s, axis=0)
# sum across accounts
x=gmv_new_df.groupby(by=dims).sum().T
# melt and calculate retained GMV
new_gmv=x.melt(ignore_index=False, value_name='NEW_GMV')
del x
# new_gmv.head()

### Churned

In [19]:
# Monthly rollup will have a '1' for each month that is considered active. This will replace these 1s with the value of the index's GMV
gmv_churn_df=yearly_churn.multiply(gmv_s, axis=0)
# sum across accounts
x=gmv_churn_df.groupby(by=dims).sum().T
# melt and calculate retained GMV
churn_gmv=x.melt(ignore_index=False, value_name='CHURNED_GMV')
del x
# churn_gmv.head()

### Combine them

In [20]:
## GMV
### Combine Them
gmv_out=retained_gmv.merge(
    churn_gmv, on=['ACTIVITY_DATE']+dims, how='left'
    ).merge(
        new_gmv, on=['ACTIVITY_DATE']+dims, how='left')
gmv_out['RETAINED GMV (NO NEW)']=gmv_out.RETAINED_GMV - gmv_out.NEW_GMV

In [21]:
gmv_out.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4430 entries, 2015-01-01 to 2024-01-01
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ACCOUNT_CLASS           4430 non-null   object 
 1   FIRST_PRODUCT_CATEGORY  4430 non-null   object 
 2   COHORT_YM               4430 non-null   object 
 3   COUNTRY                 4430 non-null   object 
 4   RETAINED_GMV            4430 non-null   float64
 5   CHURNED_GMV             4430 non-null   float64
 6   NEW_GMV                 4430 non-null   float64
 7   RETAINED GMV (NO NEW)   4430 non-null   float64
dtypes: float64(4), object(4)
memory usage: 311.5+ KB


---
## Revenue

In [22]:
## Get GMV by index 
rev_s=kpi_b.groupby(by=['ACCOUNTID']+dims).REVENUE.sum().fillna(0.00)

### Active

In [23]:
# Monthly rollup will have a '1' for each month that is considered active. This will replace these 1s with the value of the index's GMV
rev_active_df=yearly_rollup.multiply(rev_s, axis=0)
# sum across accounts
x=rev_active_df.groupby(by=dims).sum().T
# melt and calculate retained GMV
retained_rev=x.melt(ignore_index=False, value_name='RETAINED_REV')
del x

### New

In [24]:
# Monthly rollup will have a '1' for each month that is considered active. This will replace these 1s with the value of the index's GMV
rev_new_df=yearly_new.multiply(rev_s, axis=0)
# sum across accounts
x=rev_new_df.groupby(by=dims).sum().T
# melt and calculate retained GMV
new_rev=x.melt(ignore_index=False, value_name='NEW_REV')
del x

### Churned

In [25]:
# Monthly rollup will have a '1' for each month that is considered active. This will replace these 1s with the value of the index's GMV
rev_churn_df=yearly_churn.multiply(rev_s, axis=0)
# sum across accounts
x=rev_churn_df.groupby(by=dims).sum().T
# melt and calculate retained GMV
churn_rev=x.melt(ignore_index=False, value_name='CHURNED_REV')
del x

### Combine them

In [26]:
## REV
### Combine Them
rev_out=retained_rev.merge(
    churn_rev, on=['ACTIVITY_DATE']+dims, how='left'
    ).merge(
        new_rev, on=['ACTIVITY_DATE']+dims, how='left')
rev_out['RETAINED REV (NO NEW)']=rev_out.RETAINED_REV - rev_out.NEW_REV

In [27]:
new_churn_query_v2=qout.merge(
    gmv_out, on = ['ACTIVITY_DATE']+dims, how = 'left').merge(
        rev_out, on = ['ACTIVITY_DATE']+dims, how = 'left')

---
## Outputs

In [28]:
## overall summarize query
new_churn_query_v2.to_csv('NEW_CHURN_QUERY_YEARLY_V2.csv', index=True)

In [29]:
retained_y_gmv = export_by_dim(
    gmv_active_df, gb_dims=dims[2], value_name='RETAINED_GMV', out_name='nulltemp.csv')
retained_y_rev = export_by_dim(
    rev_active_df, gb_dims=dims[2], value_name='RETAINED_REV', out_name='nulltemp.csv')
retained_y = export_by_dim(
    yearly_rollup, gb_dims=dims[2], value_name='RETAINED_ACCOUNTS', out_name='nulltemp.csv')

In [30]:
retained=retained_y.merge(
    retained_y_rev, on = ['ACTIVITY_DATE',dims[2]], how = 'left').merge(
        retained_y_gmv, on = ['ACTIVITY_DATE',dims[2]], how = 'left')

retained.head()
retained.to_csv('CLOSEYR_YM_ALL-V2.csv',index=True)